In [5]:
import prettytable
import random as rnd

POPULATION_SIZE = 9
ELITE_SCHEDULES = 1
TOURNAMENT_SIZE = 3
MUTATION_RATE = 0.1

class Data:
    ROOMS = [["R1", 25], ["R2", 45], ["R3", 35]]
    MEETING_TIMES = [
        ["MT1", "MWF 09:00–10:00"],
        ["MT2", "MWF 10:00–11:00"],
        ["MT3", "TTH 09:00–10:30"],
        ["MT4", "TTH 10:30–12:00"]
    ]
    INSTRUCTORS = [
        ["T1", "Dr James Web"],
        ["T2", "Mr Mike Brown"],
        ["T3", "Dr Steve Day"],
        ["T4", "Mrs Jane Doe"]
    ]

    def __init__(self):
        self._rooms = [Room(r[0], r[1]) for r in self.ROOMS]
        self._meeting_times = [MeetingTime(mt[0], mt[1]) for mt in self.MEETING_TIMES]
        self._instructors = [Instructor(inst[0], inst[1]) for inst in self.INSTRUCTORS]

        self._courses = [
            Course("C1", "Algebra", [self._instructors[0], self._instructors[1]], 25),
            Course("C2", "Geometry", [self._instructors[0], self._instructors[1], self._instructors[2]], 35),
            Course("C3", "Calculus", [self._instructors[0], self._instructors[1]], 25),
            Course("C4", "Statistics", [self._instructors[2], self._instructors[3]], 30),
            Course("C5", "Physics", [self._instructors[3]], 35),
            Course("C6", "Biology", [self._instructors[0], self._instructors[2]], 45),
            Course("C7", "Chemistry", [self._instructors[1], self._instructors[3]], 45)
        ]

        self._depts = [
            Department("MATH", [self._courses[0], self._courses[2]]),
            Department("EE", [self._courses[1], self._courses[3], self._courses[4]]),
            Department("PHY", [self._courses[5], self._courses[6]])
        ]
        self._num_of_classes = sum(len(dept.get_courses()) for dept in self._depts)

    def get_rooms(self):
        return self._rooms

    def get_instructors(self):
        return self._instructors

    def get_courses(self):
        return self._courses

    def get_depts(self):
        return self._depts

    def get_meeting_times(self):
        return self._meeting_times

    def get_num_of_classes(self):
        return self._num_of_classes


class Schedule:
    def __init__(self, data):
        self._data = data
        self._classes = []
        self._num_of_conflicts = 0
        self._fitness = -1
        self._class_num = 0
        self._is_fitness_changed = True

    def get_classes(self):
        self._is_fitness_changed = True
        return self._classes

    def get_num_of_conflicts(self):
        return self._num_of_conflicts

    def get_fitness(self):
        if self._is_fitness_changed:
            self._fitness = self.calculate_fitness()
            self._is_fitness_changed = False
        return self._fitness

    def initialize(self):
        depts = self._data.get_depts()
        for dept in depts:
            for course in dept.get_courses():
                new_class = Class(self._class_num, dept, course)
                self._class_num += 1
                new_class.set_meeting_time(rnd.choice(self._data.get_meeting_times()))
                new_class.set_room(rnd.choice(self._data.get_rooms()))
                new_class.set_instructor(rnd.choice(course.get_instructors()))
                self._classes.append(new_class)
        # Shuffle the classes to change their sequence
        rnd.shuffle(self._classes)
        return self

    def calculate_fitness(self):
        self._num_of_conflicts = 0
        classes = self.get_classes()
        for i in range(len(classes)):
            if classes[i].get_room().get_seating_capacity() < classes[i].get_course().get_max_students():
                self._num_of_conflicts += 1
            for j in range(i + 1, len(classes)):
                if (classes[i].get_meeting_time() == classes[j].get_meeting_time() or
                        classes[i].get_instructor() == classes[j].get_instructor()) and \
                        classes[i].get_room() == classes[j].get_room():
                    self._num_of_conflicts += 1
        return 1 / (1.0 * (self._num_of_conflicts + 1))

    def __str__(self):
        classes = self.get_classes()
        table = prettytable.PrettyTable(['Class # ', 'Dept', 'Course (number, max # of students)', 'Room (Capacity)', 'Instructor', 'Meeting Time'])
        for i, cls in enumerate(classes):
            table.add_row([i, cls.get_dept().get_name(), f"{cls.get_course().get_name()} ({cls.get_course().get_number()}, {cls.get_course().get_max_students()})",
                           f"{cls.get_room().get_number()} ({cls.get_room().get_seating_capacity()})",
                           f"{cls.get_instructor().get_name()} ({cls.get_instructor().get_id()})",
                           f"{cls.get_meeting_time().get_time()} ({cls.get_meeting_time().get_id()})"])
            table.add_row(["---", "---", "---", "---", "---", "---"])  # Add a separation line after each row
        table.del_row(-1)  # Delete the last separation line
        return str(table)


class Population:
    def __init__(self, size, data):
        self._size = size
        self._schedules = [Schedule(data).initialize() for _ in range(size)]

    def get_schedules(self):
        return self._schedules


class GeneticAlgorithm:
    def evolve(self, population):
        return self.mutate_population(self.crossover_population(population))

    def crossover_population(self, pop):
        crossover_pop = Population(0, pop.get_schedules()[0]._data)
        crossover_pop.get_schedules().extend(pop.get_schedules()[:ELITE_SCHEDULES])
        for _ in range(ELITE_SCHEDULES, POPULATION_SIZE):
            schedule1 = self.select_tournament_population(pop).get_schedules()[0]
            schedule2 = self.select_tournament_population(pop).get_schedules()[0]
            crossover_pop.get_schedules().append(self.crossover_schedule(schedule1, schedule2))
        return crossover_pop

    def mutate_population(self, population):
        for i in range(ELITE_SCHEDULES, POPULATION_SIZE):
            self.mutate_schedule(population.get_schedules()[i])
        return population

    def crossover_schedule(self, schedule1, schedule2):
        crossover_schedule = Schedule(schedule1._data).initialize()
        for i in range(len(crossover_schedule.get_classes())):
            if rnd.random() > 0.5:
                crossover_schedule.get_classes()[i] = schedule1.get_classes()[i]
            else:
                crossover_schedule.get_classes()[i] = schedule2.get_classes()[i]
        return crossover_schedule

    def mutate_schedule(self, mutate_schedule):
        schedule = Schedule(mutate_schedule._data).initialize()
        for i in range(len(mutate_schedule.get_classes())):
            if MUTATION_RATE > rnd.random():
                mutate_schedule.get_classes()[i] = schedule.get_classes()[i]
        return mutate_schedule

    def select_tournament_population(self, pop):
        tournament_pop = Population(0, pop.get_schedules()[0]._data)
        for _ in range(TOURNAMENT_SIZE):
            tournament_pop.get_schedules().append(rnd.choice(pop.get_schedules()))
        tournament_pop.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
        return tournament_pop


class Course:
    def __init__(self, number, name, instructors, max_students):
        self._number = number
        self._name = name
        self._instructors = instructors
        self._max_students = max_students

    def get_name(self):
        return self._name

    def get_number(self):
        return self._number

    def get_instructors(self):
        return self._instructors

    def get_max_students(self):
        return self._max_students

    def __str__(self):
        return self._name


class Instructor:
    def __init__(self, id, name):
        self._id = id
        self._name = name

    def get_id(self):
        return self._id

    def get_name(self):
        return self._name

    def __str__(self):
        return self._name


class Room:
    def __init__(self, number, seating_capacity):
        self._number = number
        self._seating_capacity = seating_capacity

    def get_number(self):
        return self._number

    def get_seating_capacity(self):
        return self._seating_capacity


class MeetingTime:
    def __init__(self, id, time):
        self._time = time
        self._id = id

    def get_id(self):
        return self._id

    def get_time(self):
        return self._time


class Department:
    def __init__(self, name, courses):
        self._name = name
        self._courses = courses

    def get_name(self):
        return self._name

    def get_courses(self):
        return self._courses


class Class:
    def __init__(self, id, dept, course):
        self._id = id
        self._dept = dept
        self._course = course
        self._instructor = None
        self._meeting_time = None
        self._room = None

    def get_id(self):
        return self._id

    def get_dept(self):
        return self._dept

    def get_room(self):
        return self._room

    def get_course(self):
        return self._course

    def get_instructor(self):
        return self._instructor

    def get_meeting_time(self):
        return self._meeting_time

    def set_instructor(self, instructor):
        self._instructor = instructor

    def set_meeting_time(self, meeting_time):
        self._meeting_time = meeting_time

    def set_room(self, room):
        self._room = room

    def __str__(self):
        return f"{self._dept.get_name()}, {self._course.get_number()}, {self._room.get_number()}, {self._instructor.get_id()}, {self._meeting_time.get_id()}"


class DisplayMgr:
    @staticmethod
    def print_available_data(data):
        print("> All Available Data")
        DisplayMgr.print_dept(data)
        DisplayMgr.print_course(data)
        DisplayMgr.print_room(data)
        DisplayMgr.print_instructor(data)
        DisplayMgr.print_meeting_times(data)

    @staticmethod
    def print_dept(data):
        depts = data.get_depts()
        available_depts_table = prettytable.PrettyTable(['dept', 'courses'])
        for dept in depts:
            courses = dept.get_courses()
            temp_str = "[" + ', '.join(course.get_name() for course in courses) + "]"
            available_depts_table.add_row([dept.get_name(), temp_str])
        print(available_depts_table)

    @staticmethod
    def print_course(data):
        available_courses_table = prettytable.PrettyTable(['id', 'course # ', 'max # of students', 'instructors'])
        courses = data.get_courses()
        for course in courses:
            instructors = course.get_instructors()
            temp_str = ", ".join(instructor.get_name() for instructor in instructors)
            available_courses_table.add_row([course.get_number(), course.get_name(), str(course.get_max_students()), temp_str])
        print(available_courses_table)

    @staticmethod
    def print_instructor(data):
        available_instructors_table = prettytable.PrettyTable(['id', 'instructor'])
        instructors = data.get_instructors()
        for instructor in instructors:
            available_instructors_table.add_row([instructor.get_id(), instructor.get_name()])
        print(available_instructors_table)

    @staticmethod
    def print_room(data):
        available_rooms_table = prettytable.PrettyTable(['room #', 'max seating capacity'])
        rooms = data.get_rooms()
        for room in rooms:
            available_rooms_table.add_row([room.get_number(), room.get_seating_capacity()])
        print(available_rooms_table)

    @staticmethod
    def print_meeting_times(data):
        available_meeting_time_table = prettytable.PrettyTable(['id', 'Meeting Time'])
        meeting_times = data.get_meeting_times()
        for mt in meeting_times:
            available_meeting_time_table.add_row([mt.get_id(), mt.get_time()])
        print(available_meeting_time_table)

    @staticmethod
    def print_generation(population):
        table = prettytable.PrettyTable(['schedule # ', 'fitness', '# of Conflicts', 'classes [dept, class, room, instructor]'])
        for i, schedule in enumerate(population.get_schedules()):
            table.add_row([i, round(schedule.get_fitness(), 3), schedule.get_num_of_conflicts(), schedule])
        print(table)

    @staticmethod
    def print_schedule_as_table(schedule):
        classes = schedule.get_classes()
        table = prettytable.PrettyTable(['Class # ', 'Dept', 'Course (number, max # of students)', 'Room (Capacity)', 'Instructor', 'Meeting Time'])
        for i, cls in enumerate(classes):
            table.add_row([i, cls.get_dept().get_name(), f"{cls.get_course().get_name()} ({cls.get_course().get_number()}, {cls.get_course().get_max_students()})",
                           f"{cls.get_room().get_number()} ({cls.get_room().get_seating_capacity()})",
                           f"{cls.get_instructor().get_name()} ({cls.get_instructor().get_id()})",
                           f"{cls.get_meeting_time().get_time()} ({cls.get_meeting_time().get_id()})"])
            table.add_row(["---", "---", "---", "---", "---", "---"])  # Add a separation line after each row
        table.del_row(-1)  # Delete the last separation line
        print(table)


def main():
    data = Data()
    display_mgr = DisplayMgr()

    display_mgr.print_available_data(data)

    generation_number = 0
    print("\n> Generation # " + str(generation_number))
    population = Population(POPULATION_SIZE, data)
    population.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
    display_mgr.print_generation(population)
    display_mgr.print_schedule_as_table(population.get_schedules()[0])

    genetic_algorithm = GeneticAlgorithm()
    while population.get_schedules()[0].get_fitness() != 1.0:
        generation_number += 1
        print("\n> Generation # " + str(generation_number))
        population = genetic_algorithm.evolve(population)
        population.get_schedules().sort(key=lambda x: x.get_fitness(), reverse=True)
        display_mgr.print_generation(population)
        display_mgr.print_schedule_as_table(population.get_schedules()[0])


if __name__ == "__main__":
    main()


> All Available Data
+------+---------------------------------+
| dept |             courses             |
+------+---------------------------------+
| MATH |       [Algebra, Calculus]       |
|  EE  | [Geometry, Statistics, Physics] |
| PHY  |       [Biology, Chemistry]      |
+------+---------------------------------+
+----+------------+-------------------+-------------------------------------------+
| id | course #   | max # of students |                instructors                |
+----+------------+-------------------+-------------------------------------------+
| C1 |  Algebra   |         25        |        Dr James Web, Mr Mike Brown        |
| C2 |  Geometry  |         35        | Dr James Web, Mr Mike Brown, Dr Steve Day |
| C3 |  Calculus  |         25        |        Dr James Web, Mr Mike Brown        |
| C4 | Statistics |         30        |         Dr Steve Day, Mrs Jane Doe        |
| C5 |  Physics   |         35        |                Mrs Jane Doe               |
| C6 |